In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-12-02 15:00:14.335462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 15:00:14.447035: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 15:00:14.947654: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jupyter-anthony/miniconda3/envs/cxr_work/lib/python3.9/site-packages/cv2/../../l

In [2]:
from keras.applications.resnet import ResNet50, preprocess_input
HEIGHT = 256
WIDTH = 256
input_a = Input(shape=(HEIGHT, WIDTH, 3))
base_model = ResNet50(weights='imagenet', 
                    input_tensor=input_a,
                    include_top=False, 
                    input_shape=(HEIGHT, WIDTH, 3)
                    )

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(3, name='dense_logits')(x)
output = Activation('softmax', dtype='float32', name='predictions')(x)
model = Model(inputs=[input_a], outputs=[output])

2022-12-02 15:00:18.081706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 15:00:19.882119: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 196 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6
2022-12-02 15:00:19.882671: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 195 MB memory:  -> device: 1, name: GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6
2022-12-02 15:00:19.900179: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 196.44M (2059796

In [3]:
model_weight_file = 'saved_models/ResNet50_60-20-20-split_3-race_detection_LR-0.001_20221117-181956_epoch_012_val_loss_0.48259.h5'

In [4]:
model.load_weights(model_weight_file)

In [13]:
# load images
from PIL import Image
img_test = np.asarray(Image.open('data/No_finding/images/race/train/WHITE/aug_0.jpeg'))
img_test.shape

(256, 256, 3)

In [12]:
# Display
from IPython.display import Image, display
import matplotlib.cm as cm
from tensorflow import keras

In [6]:
def gradcam_img (img_array, model, last_conv_layer_name, pred_index=None, alpha=1):
    """the shape of input img_array should be [256,256,3]
    """
    img = np.expand_dims(img_array, axis=0) # add one batch_size
    
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    #print(model.output.shape)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)
    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img_array.shape[1], img_array.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img_array

    return np.array(superimposed_img), jet_heatmap

In [15]:
last_conv_layer = 'conv5_block3_3_conv' #for ResNet50
img = keras.preprocessing.image.array_to_img(gradcam_img(img_test, model, last_conv_layer, alpha=2))
plt.imshow(img)

2022-12-02 15:09:53.801832: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:433] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-12-02 15:09:53.801900: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops.cc:1152 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Exception encountered when calling layer 'conv1_conv' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} DNN library is not found. [Op:Conv2D]

Call arguments received by layer 'conv1_conv' (type Conv2D):
  • inputs=tf.Tensor(shape=(1, 262, 262, 3), dtype=float32)

In [21]:
preprocess_input = keras.applications.resnet.preprocess_input
decode_predictions = keras.applications.resnet.decode_predictions

In [19]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()
    
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))



In [20]:
img_path = '/home/jupyter-anthony/Code/CXR_MIMIC_CV/data/No_finding/images/race/train/WHITE/aug_0.jpeg'
img_array = img_test
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer, pred_index=260)
save_and_display_gradcam(img_path, heatmap)

ValueError: Input 0 of layer "model_3" is incompatible with the layer: expected shape=(None, 256, 256, 3), found shape=(256, 256, 3)

# Test

In [22]:
model_builder = keras.applications.xception.Xception
img_size = (299, 299)
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

last_conv_layer_name = "block14_sepconv2_act"

# The local path to our target image
img_path = keras.utils.get_file(
    "african_elephant.jpg", "https://i.imgur.com/Bvro0YD.png"
)

display(Image(img_path))

4217496/4217496 [==============================] - 1s 0us/step


TypeError: 'module' object is not callable

In [23]:
# Prepare image
img_array = preprocess_input(get_img_array(img_path, size=img_size))

# Make model
model = model_builder(weights="imagenet")

# Remove last layer's softmax
model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
print("Predicted:", decode_predictions(preds, top=1)[0])

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

2022-12-02 15:19:56.840989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 19.64M (20597760 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-12-02 15:19:56.842460: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 19.64M (20597760 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-12-02 15:20:06.846302: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 19.64M (20597760 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-12-02 15:20:06.849848: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 19.64M (20597760 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-12-02 15:20:06.849893: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.84MiB (rounded to 2981888)requested by op Mul
If the cause is memory fragmentation maybe the envir

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]